# Events Template

In [5]:
# Standard Libraries
import shutil
import urllib.request as urlrequest
from collections import Counter
from pathlib import Path
from pprint import pprint
from zipfile import ZipFile
import pandas as pd


In [8]:
ogd_server = 'http://opengamedata.fielddaylab.wisc.edu/'
server_data_path = 'data/AQUALAB/'
dataset_name = "AQUALAB_20230201_to_20230228"
zip_name = f"{dataset_name}_5de1a1f_events.zip"

file_url = f"{ogd_server}{server_data_path}{zip_name}"

# download the file if it doesn't exist.
if not Path(f'./{zip_name}').is_file():
    print(f"Didn't find the file {zip_name} locally, downloading from {ogd_server}...")
    !curl --ssl-no-revoke {file_url} -o {zipname}
    # with urlrequest.urlopen(ogd_server + zipname, verify=False) as remote_file, open(zipname, 'wb') as local_file:
    #     shutil.copyfileobj(remote_file, local_file)
    #     print(f"Successfully downloaded a copy of the file.")


Didn't find the file AQUALAB_20230201_to_20230228_5de1a1f_events.zip locally, downloading from http://opengamedata.fielddaylab.wisc.edu/...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: file_url
http://opengamedata.fielddaylab.wisc.edu/data/AQUALAB/AQUALAB_20230201_to_20230228_5de1a1f_events.zip


In [7]:
# Get file path and create data frame
tsv_name = f"{dataset_name}_7565f43_player-features.tsv"
survey_path = Path("./") / dataset_name / tsv_name
if not survey_path.is_file():
    !unzip ./{zipname} -d {dataset_name}
raw_data = pd.read_csv(survey_path, sep='\t')

unzip:  cannot find or open ./{zipname}, ./{zipname}.zip or ./{zipname}.ZIP.


FileNotFoundError: [Errno 2] No such file or directory: 'AQUALAB_20230201_to_20230228/AQUALAB_20230201_to_20230228_7565f43_player-features.tsv'